In [3]:
# 특성공학
import pandas as pd
df = pd.read_csv('data/Fish.csv')
perch_df = df.loc[df['Species'] == 'Perch']
perch_df.head()

,Species,Weight,Length1,Length2,Length3,Height,Width
72,Perch,5.9,7.5,8.4,8.8,2.1120,1.4080
73,Perch,32.0,12.5,13.7,14.7,3.5280,1.9992
74,Perch,40.0,13.8,15.0,16.0,3.8240,2.4320
75,Perch,51.5,15.0,16.2,17.2,4.5924,2.6316
76,Perch,70.0,15.7,17.4,18.5,4.5880,2.9415


In [13]:
# Length2, Height, Width만 사용
perch_full = perch_df[['Length2', 'Height', 'Width']]
perch_weight = perch_df[['Weight']]
perch_weight.head()

from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(perch_full, perch_weight)
train_input

# 사이킷런 변환기: 내가 가진 함수를 변환해주는 역할 -> preprocessing 전처리 활용 도구 중 특성공학 편리하게 해주는 기능인 poly 활용할 것

,Length2,Height,Width
125,43.0,12.5125,7.4165
102,27.3,8.3230,5.1373
88,22.0,6.1100,3.5250
124,43.0,11.9328,7.2772
90,22.0,5.5225,3.9950
117,39.0,11.1366,6.0030
109,32.8,10.0300,6.0180
94,23.0,5.2185,3.6260
87,22.0,5.6400,3.5250
121,40.0,12.3808,7.4624


In [17]:
from sklearn.preprocessing import PolynomialFeatures

poly = PolynomialFeatures(include_bias=False) # 1포함할지 여부 결정하는 옵션임 -> 넣어서 계산해도 문제는X
poly.fit([[3, 5]])
poly.transform([[3, 5]])
# 9: 3의 제곱, 15: 3*5, 25: 5의 제곱 -> poly.transform(): 데이터로 조합할 수 있는 여러 경우의 수를 자동으로 생성해줌
    # 원하는 데이터 집어넣기만 하면, 알아서 경우의 수 뽑아줌
# fit을 한 뒤 transform 해야 작동 -> 학습하는 역할이 아닌, 코드 작성의 일관성 유지 위해 작성하는 것

array([[ 3.,  5.,  9., 15., 25.]])

In [23]:
# perch 데이터 활용해 특성공학 하기
poly = PolynomialFeatures(include_bias=False)
poly.fit(train_input)
train_poly = poly.transform(train_input) # 높이, 길이, 두께 3개 데이터 조합으로 만들 수 있는 경우의 수를 만들어줌
train_poly.shape # 생선 42마리에 대해 9개의 특성을 만들었음
poly.get_feature_names_out() # 어떻게 연산한 9개의 특성을 만든 건지 확인

# 특성공학 바탕으로 학습시키기
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(train_poly, train_target) # 특성공학한 poly를 input 대신 넣어야 함
lr.score(train_poly, train_target) # 학습 데이터 셋으로 정답률 확인하기

# test 셋으로도 채점하기: test_poly도 만들어야 함
test_poly = poly.transform(test_input)
lr.score(test_poly, test_target)

# 특성공학 활용할 시, 지표 증가하므로 정답률 높아짐

0.9678883273348317

In [27]:
poly = PolynomialFeatures(degree=5, include_bias=False) # 몇개의 조합으로 만들지 대한 옵션(기본값은 2개) -> 높이의 5제곱 곱한 것 등 여러 특성을 만드는 것
poly.fit(train_input)
train_poly = poly.transform(train_input)
test_poly = poly.transform(test_input)
train_poly.shape
# > (42, 55) -> 특성 55개 나온 것 -> 생선 데이터보다 특성 더 많아진 것 >> 이 경우 어떤지 학습시켜보기

lr.fit(train_poly, train_target)
print(lr.score(train_poly, train_target))
print(lr.score(test_poly, test_target))
# > 0.9999999997455769
# -83.22115987225455
    # 테스트셋은 음수 나옴 -> 학습에 너무 최적화된 나머지, 채점해야 하는 데이터는 모두 틀린 것 -> 따라서 특성 많이 생성한다고 좋은 건 아님, 최적화된 특성 몇 개만 만드는 게 필요

0.9999999997455769
-83.22115987225455


In [46]:
# 데이터 정규화하기 -> 표준화하는 것
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
ss.fit(train_poly)
train_scaled = ss.transform(train_poly)
test_scaled = ss.transform(test_poly)

# 릿지: 정규화된 데이터 바탕으로 규제하기
from sklearn.linear_model import Ridge
ridge = Ridge(alpha=0.1) # alpha=1.0 : 규제의 강도 지정하는 옵션 줄 수 있음(기본값 1.0) >> 숫자 늘릴수록 규제 강하게 주겠단 것
ridge.fit(train_scaled, train_target)
print(ridge.score(train_scaled, train_target))
print(ridge.score(test_scaled, test_target))
    # 특성 55개 짜리에 규제를 적용했더니 >> 안 쓸 데이터는 규제 넣어 0에 가깝게 간소화해서 정답률 정상적으로 나옴

# 랏소
from sklearn.linear_model import Lasso
lasso = Lasso()
lasso.fit(train_scaled, train_target)
print(lasso.score(train_scaled, train_target))
print(lasso.score(test_scaled, test_target))

lasso.coef_

0.9928511802758873
0.9757653580691968
0.9904460242303096
0.9782809434869386


array([ 0.        ,  0.        ,  0.        , 13.77378528,  0.        ,
       78.63867266,  0.        ,  0.        , 59.32384889, 19.19425853,
        0.        ,  0.        , 21.20857723, 17.7263708 ,  0.        ,
       64.37582927,  6.32454751,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        , 52.9151448 ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ])